# 天猫用户复购预测-XGBoost


* Content：天猫用户复购预测,使用传统机器学习(xgboost)完成预测
* Author:  HuiHui
* Date:    2020-03-28
* Reference:
* 数据集：该数据集包含“双11”前6个月和“双11”当天匿名用户的购物日志，以及显示他们是否为重复购买者的标签信息。
    * label: 1'表示'user_id'是'merchant_id'的重复买家，而'0'则相反。'-1'表示'user_id'不是给定商家的新客户
    * activity_log: {用户id，商家id}之间的一组交互记录，其中每个记录都是一个动作，表示为“项目id:category id:brand id:time\u stamp:action\u type”#'用于分隔两个相邻元素。记录不按任何特定顺序排序

In [2]:
from google.colab import drive
drive.mount('/content/gdrive') #挂载网盘

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/RS/Repeat Buyers Prediction") #改变当前工作目录到指定的路径

In [4]:
!pip install xgboost
!pip install scikit_learn

In [5]:
import pandas as pd
import gc
import time

time1=time.time()
# 用户行为，使用format1进行加载
# 加载全量样本
user_log = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1/train_format1.csv')
submission = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1/test_format1.csv')

# # 加载小样本
# user_log = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1_small/sample_user_log.csv', dtype={'time_stamp':'str'})
# user_info = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1_small/sample_user_info.csv')
# train_data1 = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1_small/train.csv')
# submission = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format1_small/test.csv')

train_data = pd.read_csv('./Repeat Buyers Prediction DataSet/data_format2/train_format2.csv') # ???

###### 数据处理 #######
train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left') # left只保留左边的主键，只在右边主键中存在的行就不取了
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)

# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

del user_info, train_data1 #s删除变量，解除变量对数据对象的引用；del删除的是变量，而不是数据
gc.collect() #清理内存

# User特征处理
groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'}) #使用unstack（）对计数结果进行重塑;类似于转置
matrix = matrix.merge(temp, on='user_id', how='left')

# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'}) #统计全局训练数据中标签为-1的。不太懂xgboost中为什么统计这个值？？？随机？？？
matrix = matrix.merge(temp, on='merchant_id', how='left')

# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔

#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age') #get_dummies-》onehot编码,prefix表示加前缀age-
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
print(matrix)

# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

# 使用机器学习工具
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import xgboost as xgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2)

##### 使用XGBoost ######
model = xgb.XGBClassifier(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.35, #学习率调小，训练误差减小，泛化误差增大   
    seed=42 #每次产生的随机数一样；这里设置随机数种子是因为训练过程中采用了随机采样、随机属性选择   
)
model.fit(
    X_train, y_train,
    eval_metric='auc', eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10 
)

# model.fit(X_train, y_train)
prob = model.predict_proba(test_data) #预测的是标签为0/1的概率分布吗？？？标签-1呢？？？
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('./prediction_xgb.csv', index=False)

time2=time.time()
print(time2-time1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


        user_id  merchant_id label origin    u1  ...  age_7  age_8  g_0  g_1  g_2
0         34176         3906   0.0  train   451  ...      0      0    1    0    0
1         34176          121   0.0  train   451  ...      0      0    1    0    0
2         34176         4356   1.0  train   451  ...      0      0    1    0    0
3         34176         2217   0.0  train   451  ...      0      0    1    0    0
4        230784         4818   0.0  train    54  ...      0      0    1    0    0
...         ...          ...   ...    ...   ...  ...    ...    ...  ...  ...  ...
522336   228479         3111   nan   test  2004  ...      0      0    1    0    0
522337    97919         2341   nan   test    55  ...      0      1    0    1    0
522338    97919         3971   nan   test    55  ...      0      1    0    1    0
522339    32639         3536   nan   test    72  ...      0      0    1    0    0
522340    32639         3319   nan   test    72  ...      0      0    1    0    0

[522341 rows x 